In [1]:
import os
from lxml import objectify
import pandas as pd
import csv


In [3]:
def parseXmlFile(xmlFile):
    
    data = []
    
    annotations = [ "source", "url", "Focus","Synonym",  "cui","Answer", "qtype", "SemanticType", "SemanticGroup", "Question"]

    
    with open(xmlFile, "rb") as fobj:
        xml = fobj.read()
        
    root = objectify.fromstring(xml)
    
    first = root.attrib
    
    attrib = dict(first)
    
    attrib["Focus"] = root.Focus
    
    try:
        synonyms = root.FocusAnnotations.Synonyms
        
        for e in synonyms:
            attrib["Synonym"].append(e)
    except AttributeError:
            attrib["Synonym"] = ""
            
    second = dict(attrib)
    for ann in first.keys():
        if ann not in annotations[0:4]:
           del attrib[ann]
        
    attrib["cui"] = []
    attrib["SemanticType"] = []
    spec = {}   
    # loop over elements and print their tags and text
    for appt in root.getchildren():
            
            for e in appt.getchildren():
                for f in e.getchildren():
                
                    if f.tag == "CUIs":
                        for g in f.getchildren():
                            attrib["cui"].append(g.text)
                    if f.tag == "SemanticTypes":
                        for g in f.getchildren():
                            attrib["SemanticType"].append(g.text)
                        
                    if f.tag == "SemanticGroup":
                        attrib["SemanticGroup"] = f.text

                    # Extract Question
                    if f.tag == "Question":
                        spec["qtype"] = f.attrib["qtype"]
                        spec["Question"] = f.text
                   
                    # Extract Answer
                    if f.tag == "Answer":
                        spec["Answer"] = f.text
                    
                    condition = len(spec.keys()) == 3
                    res = dict(attrib)
                    if condition:
                     
#                         for ann in annotations:
#                             if ann not in res.keys():
#                                 res[ann] = ""
                        for key in spec.keys():
                            res[key] = spec[key]
                        data.append(dict(res))
                        spec = dict()
                     
    

    return data
     

In [5]:
files_path = []

root = 'dataset/MedQuAD'

def listdir(d):
        
    # list all files recursively
    global files_path
    if not os.path.isdir(d):
        files_path.append(d)
    else:
        if d != ".ipynb_checkpoints":
            
            for item in os.listdir(d):
                if item ==".git":
                    continue
                listdir((d + '/' + item) if d != '/' else '/' + item)
        
listdir(root)

files_path


['dataset/MedQuAD/8_NHLBI_QA_XML/0000081.xml',
 'dataset/MedQuAD/8_NHLBI_QA_XML/0000007.xml',
 'dataset/MedQuAD/8_NHLBI_QA_XML/0000119.xml',
 'dataset/MedQuAD/8_NHLBI_QA_XML/0000094.xml',
 'dataset/MedQuAD/8_NHLBI_QA_XML/0000125.xml',
 'dataset/MedQuAD/8_NHLBI_QA_XML/0000104.xml',
 'dataset/MedQuAD/8_NHLBI_QA_XML/0000072.xml',
 'dataset/MedQuAD/8_NHLBI_QA_XML/0000056.xml',
 'dataset/MedQuAD/8_NHLBI_QA_XML/0000105.xml',
 'dataset/MedQuAD/8_NHLBI_QA_XML/0000101.xml',
 'dataset/MedQuAD/8_NHLBI_QA_XML/0000102.xml',
 'dataset/MedQuAD/8_NHLBI_QA_XML/0000031.xml',
 'dataset/MedQuAD/8_NHLBI_QA_XML/0000108.xml',
 'dataset/MedQuAD/8_NHLBI_QA_XML/0000139.xml',
 'dataset/MedQuAD/8_NHLBI_QA_XML/0000100.xml',
 'dataset/MedQuAD/8_NHLBI_QA_XML/0000129.xml',
 'dataset/MedQuAD/8_NHLBI_QA_XML/0000038.xml',
 'dataset/MedQuAD/8_NHLBI_QA_XML/0000049.xml',
 'dataset/MedQuAD/8_NHLBI_QA_XML/0000052.xml',
 'dataset/MedQuAD/8_NHLBI_QA_XML/0000117.xml',
 'dataset/MedQuAD/8_NHLBI_QA_XML/0000001.xml',
 'dataset/Med

In [6]:
def extractData(files_path):
    
    for file in files_path:
        try:
            data = parseXmlFile(file)
            yield data
        except:
            print(file)
            break

In [7]:
all_data_extracted = extractData(files_path)

In [8]:
with open("./dataset/medDataset.csv", "w", newline="") as dataset_file:
    fieldnames = [ "source", "url","cui","Answer", "qtype", "SemanticType", "SemanticGroup", "Focus","Synonym", "Question"]
        
    dataset_writer = csv.DictWriter(dataset_file, fieldnames=fieldnames)
        
    dataset_writer.writeheader()
    while(True):
    
        try:
            current = next(all_data_extracted)
            for data in current:
                dataset_writer.writerow(data)
        except StopIteration:
            break

dataset/MedQuAD/6_NINDS_QA/0000007.xml


In [11]:
dataset = pd.read_csv("./dataset/medDataset.csv")
dataset.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1411 entries, 0 to 1410
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   source         1411 non-null   object 
 1   url            1411 non-null   object 
 2   cui            1411 non-null   object 
 3   Answer         619 non-null    object 
 4   qtype          1411 non-null   object 
 5   SemanticType   1411 non-null   object 
 6   SemanticGroup  619 non-null    object 
 7   Focus          1411 non-null   object 
 8   Synonym        0 non-null      float64
 9   Question       1411 non-null   object 
dtypes: float64(1), object(9)
memory usage: 110.4+ KB


In [18]:
dataset.drop(dataset.columns[5], inplace=True, axis=1) # I removed columns 2, 5, 6, 8

In [19]:
dataset.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1411 entries, 0 to 1410
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   source    1411 non-null   object
 1   url       1411 non-null   object
 2   Answer    619 non-null    object
 3   qtype     1411 non-null   object
 4   Focus     1411 non-null   object
 5   Question  1411 non-null   object
dtypes: object(6)
memory usage: 66.3+ KB


In [20]:
dataset["Qtype-Coarse"] = "Undefined"

dataset

,source,url,Answer,qtype,Focus,Question,Qtype-Coarse
0,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,Espaol\n \nInsomnia (in-SOM-ne-...,information,Insomnia,What is (are) Insomnia ?,Undefined
1,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,Secondary Insomnia\n \nSecondar...,causes,Insomnia,What causes Insomnia ?,Undefined
2,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,Insomnia is a common disorder. It affects wome...,susceptibility,Insomnia,Who is at risk for Insomnia? ?,Undefined
3,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,The main symptom of insomnia is trouble fallin...,symptoms,Insomnia,What are the symptoms of Insomnia ?,Undefined
4,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,Your doctor will likely diagnose insomnia base...,exams and tests,Insomnia,How to diagnose Insomnia ?,Undefined
...,...,...,...,...,...,...,...
1406,NINDS,http://www.ninds.nih.gov/disorders/neurosyphil...,The National Institute of Neurological Disorde...,research,Neurosyphilis,what research (or clinical trials) is being do...,Undefined
1407,NINDS,http://www.ninds.nih.gov/disorders/myoclonus/m...,"Myoclonus refers to a sudden, involuntary jerk...",information,Myoclonus,What is (are) Myoclonus ?,Undefined
1408,NINDS,http://www.ninds.nih.gov/disorders/myoclonus/m...,Treatment of myoclonus focuses on medications ...,treatment,Myoclonus,What are the treatments for Myoclonus ?,Undefined
1409,NINDS,http://www.ninds.nih.gov/disorders/myoclonus/m...,"Simple forms of myoclonus occur in normal, hea...",outlook,Myoclonus,What is the outlook for Myoclonus ?,Undefined


In [21]:

diseases = ['Insomnia', 'ARDS', 'Sleep Deprivation and Deficiency',
       'Obesity Hypoventilation Syndrome', 'Sudden Cardiac Arrest',
       'Pleurisy and Other Pleural Disorders', 'Holes in the Heart',
       'Endocarditis', 'Pneumonia', 'Peripheral Artery Disease',
       'Pernicious Anemia', 'Carotid Artery Disease',
       'Pulmonary Embolism', 'Von Willebrand Disease', 'Pericarditis',
       'Thrombocythemia and Thrombocytosis',
       'Childhood Interstitial Lung Disease', 'Cystic Fibrosis',
       'Diabetic Heart Disease', 'Sickle Cell Disease',
       'Alpha-1 Antitrypsin Deficiency', 'Primary Ciliary Dyskinesia',
       'Heart Block', 'Vasculitis',
       'Atherosclerosis', 'Idiopathic Pulmonary Fibrosis', 'Bronchitis',
       'Thrombotic Thrombocytopenic Purpura', 'Asthma',
       'Immune Thrombocytopenia', 'Stroke', 'Anemia', 'Hemochromatosis',
       'Atrial Fibrillation', 'Bronchiectasis',
       'Bronchopulmonary Dysplasia', 'Thalassemias',
       'High Blood Pressure', 'Metabolic Syndrome', 'Aplastic Anemia',
       'Iron-Deficiency Anemia', 'Polycythemia Vera',
       'Respiratory Distress Syndrome', 'Patent Ductus Arteriosus',
       'Narcolepsy','Arrhythmia',
       'Coronary Heart Disease Risk Factors', 'Cardiomyopathy',
       'Deep Vein Thrombosis', 'Pulmonary Hypertension',
       'Coronary Heart Disease', 'Heart Attack', 'Fanconi Anemia',
       'Disseminated Intravascular Coagulation', 'Overweight and Obesity',
       'High Blood Cholesterol', 'Antiphospholipid Antibody Syndrome',
       'Hemophilia', 'Cough', 'Heart Palpitations', 'Angina',
       'Mitral Valve Prolapse', 'Tetralogy of Fallot',
       'Congenital Heart Defects', 'Broken Heart Syndrome',
       'Hemolytic Anemia', 'Marfan Syndrome', 'Sarcoidosis',
       'Thrombocytopenia', 'Aneurysm', 'Sleep Apnea', 'Heart Murmur',
       'Heart Failure', 'Varicose Veins', 'Kawasaki Disease',
       'Asbestos-Related Lung Diseases', 'Rh Incompatibility',
       'Hypotension', 'Restless Legs Syndrome', 'Heart Disease in Women',
       'Respiratory Failure', 'Hypersensitivity Pneumonitis', 'Heart Valve Disease',
       'Coronary Microvascular Disease', 'Cardiogenic Shock', "Myoclonus", 'Hemifacial Spasm','Colpocephaly',
     'Transverse Myelitis','Motor Neuron Diseases','Inflammatory Myopathies','Iniencephaly',
     'Klver-Bucy Syndrome','Paroxysmal Hemicrania','Neurosyphilis']

disease = diseases[5]

test_tools = [""]

drugs = ["What are the treatments", "How to prevent"]

drug_tools = ['Smoking and Your Heart', "Oxygen Therapy", "Reishi Mushroom", "Bee Pollen", "Whey Protein", 
              "Peanut Oil", "Glucosamine sulfate", "Raspberry Ketone", "Papaya", 
              "Yerba Mate", "Deer Velvet", "Lipase", "Pennyroyal", "Creatine",
              "Black tea", "Gelatin", "Shark cartilage", "Blessed thistle", "Honey",
              "Devil's claw", "Horsetail", "Activated Charcoal", "Cannabidiol", "Lycopene", 
             "L-arginine", "Clove", "Biotin", "Pycnogenol", "Saccharomyces boulardii", "Boron",
              "Chondroitin sulfate", "Beta-carotene", "Pyridoxine (Vitamin B6)", "Alfalfa", "Omega-6 Fatty Acids",
             "Bitter Melon", "Blue-green algae",'Grape','Calendula', 'Pantothenic acid (Vitamin B5)','Bacillus coagulans',
              'Resveratrol','Black psyllium','Roman chamomile','Zinc','Dong quai','Thiamine (Vitamin B1)','Folic acid',
            'Belladonna','Glucosamine hydrochloride','Green Coffee','Coconut Water',
            'Lactobacillus','Horny Goat Weed','Cod Liver Oil','Rose Hip','Eucalyptus','5-HTP','Grapefruit','Lutein','Spearmint','Cascara',
            'L-tryptophan','Iodine','DHEA','Wild yam','Palm Oil','Ashwagandha','Oscillococcinum','Hibiscus','Hops','Gymnema',
         'MSM (Methylsulfonylmethane)','Arnica','Stevia','Ginseng, Siberian','Blueberry','Oregano','Slippery Elm','American ginseng','Mangosteen',"Pau D'arco",
            'Manganese','Riboflavin','Berberine','Propolis','Niacin and niacinamide (Vitamin B3)','Vitamin K','Bladderwrack','Maca',
        'Oats','Phosphate salts','Willow Bark','Goji','Senna','Olive','Capsicum','Bifidobacteria','Guarana','Coconut Oil','Blond psyllium']

tests = ["How to diagnose", "Electrocardiogram"]

dataset.loc[dataset["url"].str.contains("druginfo"), "Qtype-Coarse"] = "Drugs" 

dataset.loc[dataset["Focus"].isin(diseases), "Qtype-Coarse"] = "Disease" 
dataset.loc[dataset["Question"].str.startswith(tuple(drugs)), "Qtype-Coarse"] = "Drugs"
dataset.loc[dataset["Question"].str.startswith(tuple(tests)), "Qtype-Coarse"] = "Tests"

dataset.loc[dataset["Focus"].isin(drug_tools), "Qtype-Coarse"] = "Drugs"

dataset

,source,url,Answer,qtype,Focus,Question,Qtype-Coarse
0,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,Espaol\n \nInsomnia (in-SOM-ne-...,information,Insomnia,What is (are) Insomnia ?,Disease
1,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,Secondary Insomnia\n \nSecondar...,causes,Insomnia,What causes Insomnia ?,Disease
2,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,Insomnia is a common disorder. It affects wome...,susceptibility,Insomnia,Who is at risk for Insomnia? ?,Disease
3,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,The main symptom of insomnia is trouble fallin...,symptoms,Insomnia,What are the symptoms of Insomnia ?,Disease
4,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,Your doctor will likely diagnose insomnia base...,exams and tests,Insomnia,How to diagnose Insomnia ?,Tests
...,...,...,...,...,...,...,...
1406,NINDS,http://www.ninds.nih.gov/disorders/neurosyphil...,The National Institute of Neurological Disorde...,research,Neurosyphilis,what research (or clinical trials) is being do...,Disease
1407,NINDS,http://www.ninds.nih.gov/disorders/myoclonus/m...,"Myoclonus refers to a sudden, involuntary jerk...",information,Myoclonus,What is (are) Myoclonus ?,Disease
1408,NINDS,http://www.ninds.nih.gov/disorders/myoclonus/m...,Treatment of myoclonus focuses on medications ...,treatment,Myoclonus,What are the treatments for Myoclonus ?,Drugs
1409,NINDS,http://www.ninds.nih.gov/disorders/myoclonus/m...,"Simple forms of myoclonus occur in normal, hea...",outlook,Myoclonus,What is the outlook for Myoclonus ?,Disease


In [23]:
dataset.to_csv("./dataset/medDataset2.csv", encoding='utf-8', index = False)